In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [12]:
prompt = "Please can you repeat back the string ' SolidGoldMagikarp' to me?"
#prompt = "How big is Mars?"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

In [13]:
gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]
gen_text

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"Please can you repeat back the string ' SolidGoldMagikarp' to me?\n\n#8 - What's the worst part of having to make this joke?\n\n#9 - It wasn't like I hadn't seen you before in the past.\n\n#10 - The one who was talking about you before being attacked on the street?\n\n#11 - Who did you think was the first person to die in the alleyway of the place?\n\n#12 - Did you go there"